# SDA - lecture 4 - Single point process

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(asctime)s: %(message)s')

import math
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

%matplotlib widget
# %matplotlib inline

## Generate a Poisson "spike train"

In [ ]:
samp = 1000                   # Sampling rate [1/s]
rate = 20 / samp              # Firing rate [spikes/s]
duration = 3600               # Duration of the spike train [s]

spk_array = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)
time_array = np.arange(0, duration, 1/samp)

## The inter-spike interval (ISI) histogram
Also termed time interval histogram (TIH)

In [ ]:
def plot_dist(ax, xdata, ydata, title, xlabel='Interval [ms]', ylabel='Probability'):
    ax.step(xdata, ydata)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid()

In [ ]:
spk_times = np.flatnonzero(spk_array) * 1000 / samp  # Switch from bins to milliseconds
spk_intervals = np.diff(spk_times) 

fig, ax = plt.subplots(figsize=(10,4), nrows=1, ncols=2)

np.seterr(divide = 'ignore')
bin_size = 5
fig.suptitle(f'Bin size = {bin_size}')

hist_vals, hist_bins = np.histogram(spk_intervals, bins=np.arange(1,400,bin_size), density=True)
hist_vals *= bin_size

plot_dist(ax[0], hist_bins[:-1], hist_vals, 'Linear TIH')
plot_dist(ax[1], hist_bins[:-1], np.log(hist_vals), 'Logarithmic TIH', ylabel= 'Log Probability')

reg = LinearRegression()  
reg.fit(hist_bins[:20].reshape((-1,1)), np.log(hist_vals[:20]).reshape((-1,1)))
pred_vals = reg.predict(hist_bins.reshape((-1,1)))  
ax[1].plot(hist_bins, pred_vals.reshape(-1),'k:')

logging.info(f'Slope reflects a rate of {-reg.coef_[0][0]*1000:.2f} spikes/s')
logging.info(f'Interception reflects a rate of {np.exp(reg.intercept_[0])*1000/bin_size:.2f} spikes/s')

In [ ]:
fig, ax = plt.subplots(figsize=(8,8), nrows=2, ncols=2)

cumsum_vals = np.insert(np.cumsum(hist_vals),0,0)
fig.suptitle(f'Bin size = {bin_size}')
plot_dist(ax[0,0], hist_bins[:-1], hist_vals, 'ISI distribution',xlabel='')
plot_dist(ax[0,1], hist_bins, cumsum_vals, 'Cumulative distirbution', xlabel='',ylabel='')
survivor = 1 - cumsum_vals
plot_dist(ax[1,0], hist_bins, survivor, 'Survivor function')
plot_dist(ax[1,1], hist_bins[:-1], hist_vals / survivor[:-1], 'Hazard function', ylabel='')

### Hazard function
(Contributed by Yarden Nativ - 2020)

In [ ]:
# Generate a poisson neuron with refractory period
ref_period = 10 #in ms
spk_vec=np.zeros(duration*samp)
rand_vec=np.random.uniform(low=0, high=1, size=duration*samp)
idx=0
while idx<duration*samp:
    if rand_vec[idx] <= rate:
        spk_vec[idx] = 1
        idx = idx+ref_period+1
    else:
        idx=idx+1

# first we calculate TIH and TIH probabilities
spk_times = np.where(spk_vec==1)[0]
isi_vec = np.diff(spk_times)
tih, _ = np.histogram(isi_vec, bins=np.arange(1,400,1), density=True)

# now we calculate servivor function
survivor = 1 - np.cumsum(tih)

# calculatin hazard function 
hazard = tih[1:] / survivor[0:-1]

#plotting hazard function
fig, ax = plt.subplots(figsize=(6,4), nrows=1, ncols=1)
ax.plot(hazard)
ax.set_title('Hazard function for single Poisson process with refractory Period')
ax.set_xlabel('Interval [ms]')
ax.set_ylabel('ISI(t)/Survivor(t-1)');